# Fusion 360 Gewindetool – XML-Editor mit Jupyter Notebook

Dieses Projekt hilft dir, benutzerdefinierte Gewinde für Autodesk Fusion 360 einfach in eine XML-Datei einzutragen.
Es verwendet ein interaktives Jupyter Notebook mit `ipywidgets`, um Eingaben auf Deutsch zu erfassen und korrekt in das englischsprachige XML-Format zu übertragen.

In [39]:
from IPython.display import display, HTML
import ipywidgets as widgets
from ipywidgets import Layout, HBox, VBox
import xml.etree.ElementTree as ET
import subprocess
import datetime
import threading
import time
import xml.dom.minidom

# CSS hacks
display(HTML("<style>.highlighted select { border: 3px solid lightgreen !important; }</style>"))
headline = widgets.HTML(value="<h1>Fusion 360 Gewindetool – XML-Editor mit Jupyter Notebook</h1>")
display(headline)


# Feste XML-Datei
xml_path = 'data/AstroISOmetric.xml'

def remove_blank_lines(xml_string):
    lines = xml_string.split('\n')
    return '\n'.join([line for line in lines if line.strip() != ''])

def pretty_print_xml(path):
    with open(path, 'r') as f:
        xml_string = f.read()
    parsed = xml.dom.minidom.parseString(xml_string)
    with open(path, 'w') as f:
        f.write(parsed.toprettyxml(indent="  "))

def load_dropdown_options():
    tree = ET.parse(xml_path)
    root = tree.getroot()
    options = ['<Neu laden>']
    for thread_size in root.findall('ThreadSize'):
        designation = thread_size.find('Designation/ThreadDesignation').text
        if designation:
            options.append(designation)
    thread_dropdown.options = options

def delete_thread_from_xml(designation_en):
    tree = ET.parse(xml_path)
    root = tree.getroot()
    for ts in root.findall('ThreadSize'):
        if ts.find('Designation/ThreadDesignation').text == designation_en:
            root.remove(ts)
            tree.write(xml_path)
            print(f"🗑️ Gewinde {designation_en} gelöscht.")
            return
    print(f"⚠️ Gewinde {designation_en} nicht gefunden.")

def on_delete_clicked(b):
    delete_thread_from_xml(thread_dropdown.value)
    load_dropdown_options()
    thread_dropdown.value = '<Neu laden>'

def fill_fields_from_selection(change):
    selection = change['new']
    if selection == '<Neu laden>':
        for w in all_widgets:
            w.value = '' if isinstance(w, widgets.Text) else 0
        return
    tree = ET.parse(xml_path)
    root = tree.getroot()
    for thread_size in root.findall('ThreadSize'):
        if thread_size.find('Designation/ThreadDesignation').text == selection:
            size.value = float(thread_size.find('Size').text)
            designation_de.value = thread_size.find('Designation/CTD').text
            designation_en.value = selection
            pitch.value = float(thread_size.find('Designation/Pitch').text)
            for t in thread_size.find('Designation').findall('Thread'):
                gender = t.find('Gender').text
                cls = t.find('Class').text
                major = float(t.find('MajorDia').text)
                pitch_dia = float(t.find('PitchDia').text)
                minor = float(t.find('MinorDia').text)
                if gender == 'external' and cls == '6g':
                    major_dia_ext.value, pitch_dia_ext.value, minor_dia_ext.value = major, pitch_dia, minor
                elif gender == 'internal' and cls == '6H':
                    major_dia_int.value, pitch_dia_int.value, minor_dia_int.value = major, pitch_dia, minor
                    tap_drill.value = float(t.find('TapDrill').text)
                elif gender == 'external' and cls == '4g6g':
                    major_dia_4g6g.value, pitch_dia_4g6g.value, minor_dia_4g6g.value = major, pitch_dia, minor
            break

import xml.dom.minidom

def remove_blank_lines(xml_string):
    lines = xml_string.split('\n')
    non_empty_lines = [line for line in lines if line.strip() != '']
    return '\n'.join(non_empty_lines)

def add_thread_to_xml(path, size, designation_de, designation_en, pitch,
                      major_dia_ext, pitch_dia_ext, minor_dia_ext,
                      major_dia_int, pitch_dia_int, minor_dia_int,
                      major_dia_4g6g, pitch_dia_4g6g, minor_dia_4g6g,
                      tap_drill):
    tree = ET.parse(path)
    root = tree.getroot()

    for ts in root.findall('ThreadSize'):
        desig = ts.find('Designation/ThreadDesignation').text
        if desig == designation_en:
            root.remove(ts)

    thread_size = ET.SubElement(root, 'ThreadSize')
    ET.SubElement(thread_size, 'Size').text = str(size)
    designation_el = ET.SubElement(thread_size, 'Designation')
    ET.SubElement(designation_el, 'ThreadDesignation').text = designation_en
    ET.SubElement(designation_el, 'CTD').text = designation_de
    ET.SubElement(designation_el, 'Pitch').text = str(pitch)

    thread_ext = ET.SubElement(designation_el, 'Thread')
    ET.SubElement(thread_ext, 'Gender').text = 'external'
    ET.SubElement(thread_ext, 'Class').text = '6g'
    ET.SubElement(thread_ext, 'MajorDia').text = str(major_dia_ext)
    ET.SubElement(thread_ext, 'PitchDia').text = str(pitch_dia_ext)
    ET.SubElement(thread_ext, 'MinorDia').text = str(minor_dia_ext)

    thread_int = ET.SubElement(designation_el, 'Thread')
    ET.SubElement(thread_int, 'Gender').text = 'internal'
    ET.SubElement(thread_int, 'Class').text = '6H'
    ET.SubElement(thread_int, 'MajorDia').text = str(major_dia_int)
    ET.SubElement(thread_int, 'PitchDia').text = str(pitch_dia_int)
    ET.SubElement(thread_int, 'MinorDia').text = str(minor_dia_int)
    ET.SubElement(thread_int, 'TapDrill').text = str(tap_drill)

    thread_ext_4g6g = ET.SubElement(designation_el, 'Thread')
    ET.SubElement(thread_ext_4g6g, 'Gender').text = 'external'
    ET.SubElement(thread_ext_4g6g, 'Class').text = '4g6g'
    ET.SubElement(thread_ext_4g6g, 'MajorDia').text = str(major_dia_4g6g)
    ET.SubElement(thread_ext_4g6g, 'PitchDia').text = str(pitch_dia_4g6g)
    ET.SubElement(thread_ext_4g6g, 'MinorDia').text = str(minor_dia_4g6g)

    tree.write(path)

    # Schön formatieren ohne Leerzeilen
    with open(path, 'r') as f:
        xml_str = f.read()
    pretty_xml = xml.dom.minidom.parseString(xml_str).toprettyxml(indent="  ")
    pretty_xml_clean = remove_blank_lines(pretty_xml)
    with open(path, 'w') as f:
        f.write(pretty_xml_clean)

    print(f"✅ Gewinde {designation_en} ({designation_de}) erfolgreich in {path} gespeichert und formatiert.")
    
# --- Widgets und Layouts ---    
field_layout = Layout(width='500px')
vbox_layout = Layout(width='550px', margin='0 40px 20px 0')
style = {'description_width': '350px'}

# Widgets für die Eingabefelder
thread_dropdown = widgets.Dropdown(description='Vorhandene Gewinde:', options=['<Neu laden>'], layout=field_layout, style=style)
size = widgets.FloatText(description='Nenndurchmesser (Size):', layout=field_layout, style=style)
designation_de = widgets.Text(description='Custom Thread designation:', layout=field_layout, style=style)
designation_en = widgets.Text(description='Technical designation:', layout=field_layout, style=style)
pitch = widgets.FloatText(description='Steigung (Pitch):', layout=field_layout, style=style)


# Widgets für die Gewinde-Parameter
label_ext_6g = widgets.HTML(value='<div style="text-align:right;"><b>External 6g</b></div>')
major_dia_ext, pitch_dia_ext, minor_dia_ext = [widgets.FloatText(description=d, layout=field_layout, style=style)
                                               for d in ['MajorDia:', 'PitchDia:', 'MinorDia:']]

label_int_6H = widgets.HTML(value='<div style="text-align:right;"><b>Internal 6H</b></div>')
major_dia_int, pitch_dia_int, minor_dia_int, tap_drill = [widgets.FloatText(description=d, layout=field_layout, style=style)
                                                          for d in ['MajorDia:', 'PitchDia:', 'MinorDia:', 'TapDrill:']]

label_ext_4g6g = widgets.HTML(value='<div style="text-align:right;"><b>External 4g6g</b></div>')
major_dia_4g6g, pitch_dia_4g6g, minor_dia_4g6g = [widgets.FloatText(description=d, layout=field_layout, style=style)
                                                  for d in ['MajorDia:', 'PitchDia:', 'MinorDia:']]

# Button-Widgets
save_button = widgets.Button(description='Speichern', style=style)
save_new_button = widgets.Button(description='Als neu speichern', style=style)
delete_button = widgets.Button(description='Löschen', style=style)

# Save-Buttons initial deaktivieren
save_button.disabled = True
save_new_button.disabled = True
delete_button.disabled = True

def on_save_clicked(b):
    add_thread_to_xml(xml_path, size.value, designation_de.value, designation_en.value, pitch.value,
                      major_dia_ext.value, pitch_dia_ext.value, minor_dia_ext.value,
                      major_dia_int.value, pitch_dia_int.value, minor_dia_int.value,
                      major_dia_4g6g.value, pitch_dia_4g6g.value, minor_dia_4g6g.value,
                      tap_drill.value)
    load_dropdown_options()
    thread_dropdown.value = designation_en.value
    save_button.style.button_color = 'lightgreen'
    save_new_button.style.button_color = ''
    delete_button.style.button_color = ''

def on_save_new_clicked(b):
    # Neue eindeutige Bezeichnung erzeugen (falls mehrfach geklickt)
    base_name = designation_en.value.rstrip('_neu')
    counter = 1
    new_designation = f"{base_name}_neu"
    existing_designations = [opt for opt in thread_dropdown.options if opt != '<Neu laden>']
    while new_designation in existing_designations:
        new_designation = f"{base_name}_neu{counter}"
        counter += 1
    designation_en.value = new_designation
    add_thread_to_xml(xml_path, size.value, designation_de.value, designation_en.value, pitch.value,
                      major_dia_ext.value, pitch_dia_ext.value, minor_dia_ext.value,
                      major_dia_int.value, pitch_dia_int.value, minor_dia_int.value,
                      major_dia_4g6g.value, pitch_dia_4g6g.value, minor_dia_4g6g.value,
                      tap_drill.value)
    load_dropdown_options()
    thread_dropdown.value = designation_en.value
    save_new_button.style.button_color = 'lightgreen'
    save_button.style.button_color = ''
    delete_button.style.button_color = ''

save_button.on_click(on_save_clicked)
save_new_button.on_click(on_save_new_clicked)
delete_button.on_click(on_delete_clicked)

button_box = VBox([save_button, save_new_button, delete_button])

all_widgets = [size, designation_de, designation_en, pitch,
               major_dia_ext, pitch_dia_ext, minor_dia_ext,
               major_dia_int, pitch_dia_int, minor_dia_int, tap_drill,
               major_dia_4g6g, pitch_dia_4g6g, minor_dia_4g6g]

top_row = HBox([VBox([thread_dropdown, size, designation_de, designation_en, pitch], layout=vbox_layout), button_box])

# Lade Dropdown mit Fehlerbehandlung
try:
    load_dropdown_options()
except Exception as e:
    print(f"⚠️ Fehler beim Laden der Dropdown-Optionen: {e}")
    thread_dropdown.options = ['<Neu laden>']

# Setze Dropdown auf Startwert
thread_dropdown.value = '<Neu laden>'
thread_dropdown.observe(fill_fields_from_selection, names='value')



def on_export_clicked(b):
    subprocess.run(['./bin/sync_xml.sh', 'export'])
    status_label.value = f"✅ Export aus Fusion abgeschlossen am {datetime.datetime.now().strftime('%d.%m.%Y %H:%M:%S')}"
    save_button.disabled = False
    save_new_button.disabled = False
    delete_button.disabled = False
    save_button.style.button_color = 'lightgreen'
    save_new_button.style.button_color = 'lightgreen'
    delete_button.style.button_color = 'lightgreen'
    thread_dropdown.add_class('highlighted')
    export_button.style.button_color = None  # Zurück auf neutral/grau
    
    
def on_import_clicked(b):
    subprocess.run(['./bin/sync_xml.sh', 'import'])
    last_import_time[0] = datetime.datetime.now()
    update_status_label()
    status_label.value = f"✅ Import in Fusion abgeschlossen am {last_import_time[0].strftime('%d.%m.%Y %H:%M:%S')}"
    save_button.style.button_color = 'lightgreen'
    save_new_button.style.button_color = 'lightgreen'
    delete_button.style.button_color = 'lightgreen'
    thread_dropdown.add_class('highlighted')
    
def on_venv_clicked(b):
    subprocess.run(['./bin/create_venv.sh'])
    status_label.value = f"✅ .venv neu erstellt: {datetime.datetime.now().strftime('%d.%m.%Y %H:%M:%S')}"

# --- Neue Buttons für Export, Import, venv ---
export_button = widgets.Button(description='Get XML from Fusion')
import_button = widgets.Button(description='Patch Fusion XML')
venv_button = widgets.Button(description='Rebuild .venv')
status_label = widgets.Label(value='⏳ Noch kein Export oder Import ausgeführt.')
control_buttons = HBox([export_button, import_button, venv_button])

export_button.on_click(on_export_clicked)
import_button.on_click(on_import_clicked)
venv_button.on_click(on_venv_clicked)

# Buttons initial auf Grün (Get XML) setzen
export_button.style.button_color = 'lightgreen'


# Sprachumschalter und Übersetzungen
current_language = {'lang': 'EN'}

labels = {
    'DE': {
        'headline': 'Fusion 360 Gewindetool – XML-Editor mit Jupyter Notebook',
        'thread_dropdown': 'Vorhandene Gewinde:',
        'size': 'Nenndurchmesser (Size):',
        'designation_de': 'Custom Thread designation - Freitext (CTD):',
        'designation_en': 'Technische Bezeichnung (ThreadDesignation) z.B. M56x0.75:',
        'pitch': 'Steigung (Pitch):',
        'ext_6g': '<b>External 6g</b>',
        'int_6H': '<b>Internal 6H</b>',
        'ext_4g6g': '<b>External 4g6g</b>',
        'save': 'Speichern',
        'save_new': 'Als neu speichern',
        'delete': 'Löschen',
        'export': 'Get XML from Fusion',
        'import': 'Patch Fusion XML',
        'venv': 'Rebuild .venv',
        'status': '⏳ Noch kein Export oder Import ausgeführt.'
    },
    'EN': {
        'headline': 'Fusion 360 Thread Tool – XML Editor with Jupyter Notebook',
        'thread_dropdown': 'Existing Threads:',
        'size': 'Nominal Diameter (Size):',
        'designation_de': 'Custom Thread designation (free text, CTD):',
        'designation_en': 'Technical designation (ThreadDesignation) e.g. M56x0.75:',
        'pitch': 'Pitch:',
        'ext_6g': '<b>External 6g</b>',
        'int_6H': '<b>Internal 6H</b>',
        'ext_4g6g': '<b>External 4g6g</b>',
        'save': 'Save',
        'save_new': 'Save as New',
        'delete': 'Delete',
        'export': 'Get XML from Fusion',
        'import': 'Patch Fusion XML',
        'venv': 'Rebuild .venv',
        'status': '⏳ No export or import done yet.'
    }
}

def update_labels():
    lang = current_language['lang']
    headline.value = f"<h1>{labels[lang]['headline']}</h1>"
    thread_dropdown.description = labels[lang]['thread_dropdown']
    size.description = labels[lang]['size']
    designation_de.description = labels[lang]['designation_de']
    designation_en.description = labels[lang]['designation_en']
    pitch.description = labels[lang]['pitch']
    label_ext_6g.value = labels[lang]['ext_6g']
    label_int_6H.value = labels[lang]['int_6H']
    label_ext_4g6g.value = labels[lang]['ext_4g6g']
    save_button.description = labels[lang]['save']
    save_new_button.description = labels[lang]['save_new']
    delete_button.description = labels[lang]['delete']
    export_button.description = labels[lang]['export']
    import_button.description = labels[lang]['import']
    venv_button.description = labels[lang]['venv']
    status_label.value = labels[lang]['status']
    
toggle_lang_button = widgets.Button(description='🇩🇪 Auf Deutsch umschalten')

def on_toggle_lang_clicked(b):
    if current_language['lang'] == 'EN':
        current_language['lang'] = 'DE'
        toggle_lang_button.description = '🇬🇧 Switch to English'
    else:
        current_language['lang'] = 'EN'
        toggle_lang_button.description = '🇩🇪 Auf Deutsch umschalten'
    update_labels()

toggle_lang_button.on_click(on_toggle_lang_clicked)


# Letzter Importzeitpunkt
last_import_time = [None]

# Sprachumschalter anzeigen
display(toggle_lang_button)

# obere Eingabezeile und Button-Block
display(top_row)

# Anzeige-Teil
display(
    control_buttons,
    status_label,
)

# Gewinde-Bereiche anzeigen
display(
    label_ext_6g, major_dia_ext, pitch_dia_ext, minor_dia_ext,
    label_int_6H, major_dia_int, pitch_dia_int, minor_dia_int, tap_drill,
    label_ext_4g6g, major_dia_4g6g, pitch_dia_4g6g, minor_dia_4g6g
)

# Unsichtbarer Spacer am Ende der App
spacer = widgets.Box(layout=widgets.Layout(height='30px'))
display(spacer)

update_labels()




HTML(value='<h1>Fusion 360 Gewindetool – XML-Editor mit Jupyter Notebook</h1>')

Button(description='🇩🇪 Auf Deutsch umschalten', style=ButtonStyle())

Label(value='⏳ Noch kein Export oder Import ausgeführt.')

HTML(value='<div style="text-align:right;"><b>External 6g</b></div>')

FloatText(value=0.0, description='MajorDia:', layout=Layout(width='500px'), style=DescriptionStyle(description…

...

HTML(value='<div style="text-align:right;"><b>External 4g6g</b></div>')

FloatText(value=0.0, description='MajorDia:', layout=Layout(width='500px'), style=DescriptionStyle(description…

FloatText(value=0.0, description='PitchDia:', layout=Layout(width='500px'), style=DescriptionStyle(description…

FloatText(value=0.0, description='MinorDia:', layout=Layout(width='500px'), style=DescriptionStyle(description…

Box(layout=Layout(height='30px'))